In [0]:

# Create a dataframe from a static list
# syntax: spark.createDataFrame(data =  , schema = )

from pyspark.sql.types import *


staticlist = [(" raja", "pushpa", "USA","",30000),
            ("        priya", "     pushpa", "USA","",29900),
            (" Karthik",       "Subu", "USA","CA",6000),
            (" James", "Smith            ", "USA","FL",20000),
            ("Martin", "Jones", "         USA","CA",3000),
            ("Sam", "Anderson        ", "UK","LND",8000),
            ("      Maria", "       Patrick", "UK","MCR",7000),
            ("Robet", "Bevon", "UK","LND",3500),
            ("Maria", "         Anderson", "UK","MCR",3000)
            ]
columns = ["firstname","lastname","country","state","sal"]            
df = spark.createDataFrame( data = staticlist, schema = columns)
df.show()

+-------------+-----------------+------------+-----+-----+
|    firstname|         lastname|     country|state|  sal|
+-------------+-----------------+------------+-----+-----+
|         raja|           pushpa|         USA|     |30000|
|        priya|           pushpa|         USA|     |29900|
|      Karthik|             Subu|         USA|   CA| 6000|
|        James|Smith            |         USA|   FL|20000|
|       Martin|            Jones|         USA|   CA| 3000|
|          Sam| Anderson        |          UK|  LND| 8000|
|        Maria|          Patrick|          UK|  MCR| 7000|
|        Robet|            Bevon|          UK|  LND| 3500|
|        Maria|         Anderson|          UK|  MCR| 3000|
+-------------+-----------------+------------+-----+-----+



In [0]:
#changing the datatype of the column

# option1: using select method

from pyspark.sql.types import *


df_cast1 = df.select(
    "firstname",
    "lastname",
    "country",
    "state",
    df.sal.cast(IntegerType())
)
df_cast1.printSchema()


root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- sal: integer (nullable = true)



In [0]:
#option 2 :using the withcolumn method
df_cast2 = df.withColumn("bonus",df.sal.cast(IntegerType()))
df_cast2.printSchema()


root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- sal: long (nullable = true)
 |-- bonus: integer (nullable = true)



In [0]:
#Adding a new calculated columns using the withcolumn method
# add a new bonus column with double the current salary
df1 = df.withColumn("Bonus",df.sal*2)
df1.show()

+-------------+-----------------+------------+-----+-----+-----+
|    firstname|         lastname|     country|state|  sal|Bonus|
+-------------+-----------------+------------+-----+-----+-----+
|         raja|           pushpa|         USA|     |30000|60000|
|        priya|           pushpa|         USA|     |29900|59800|
|      Karthik|             Subu|         USA|   CA| 6000|12000|
|        James|Smith            |         USA|   FL|20000|40000|
|       Martin|            Jones|         USA|   CA| 3000| 6000|
|          Sam| Anderson        |          UK|  LND| 8000|16000|
|        Maria|          Patrick|          UK|  MCR| 7000|14000|
|        Robet|            Bevon|          UK|  LND| 3500| 7000|
|        Maria|         Anderson|          UK|  MCR| 3000| 6000|
+-------------+-----------------+------------+-----+-----+-----+



In [0]:
# add a new calculatred column full name, concatenate fir and last name

from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import *
from delta.tables import *


In [0]:

df1 = df.withColumn("FullName",concat(df.firstname,lit(" "),df.lastname))
df1.show()

+-------------+-----------------+------------+-----+-----+--------------------+
|    firstname|         lastname|     country|state|  sal|            FullName|
+-------------+-----------------+------------+-----+-----+--------------------+
|         raja|           pushpa|         USA|     |30000|         raja pushpa|
|        priya|           pushpa|         USA|     |29900|        priya    ...|
|      Karthik|             Subu|         USA|   CA| 6000|        Karthik Subu|
|        James|Smith            |         USA|   FL|20000| James Smith     ...|
|       Martin|            Jones|         USA|   CA| 3000|        Martin Jones|
|          Sam| Anderson        |          UK|  LND| 8000|Sam Anderson        |
|        Maria|          Patrick|          UK|  MCR| 7000|      Maria      ...|
|        Robet|            Bevon|          UK|  LND| 3500|         Robet Bevon|
|        Maria|         Anderson|          UK|  MCR| 3000|Maria          An...|
+-------------+-----------------+-------

In [0]:
# add a static column called reportname and values shoud be "empinfo"
df1 = df.withColumn("ReportNam",lit("Empinfo"))
df1.show()

+-------------+-----------------+------------+-----+-----+---------+
|    firstname|         lastname|     country|state|  sal|ReportNam|
+-------------+-----------------+------------+-----+-----+---------+
|         raja|           pushpa|         USA|     |30000|  Empinfo|
|        priya|           pushpa|         USA|     |29900|  Empinfo|
|      Karthik|             Subu|         USA|   CA| 6000|  Empinfo|
|        James|Smith            |         USA|   FL|20000|  Empinfo|
|       Martin|            Jones|         USA|   CA| 3000|  Empinfo|
|          Sam| Anderson        |          UK|  LND| 8000|  Empinfo|
|        Maria|          Patrick|          UK|  MCR| 7000|  Empinfo|
|        Robet|            Bevon|          UK|  LND| 3500|  Empinfo|
|        Maria|         Anderson|          UK|  MCR| 3000|  Empinfo|
+-------------+-----------------+------------+-----+-----+---------+



In [0]:
#rename a column
#option 1: using the select method
df1 = df.select(df.firstname.alias("F_N"),
                df.lastname.alias("L_N"),
                df.state,df.country.alias("CN"),df.sal
                )
df1.show()                

+-------------+-----------------+------------+
|          F_N|              L_N|          CN|
+-------------+-----------------+------------+
|         raja|           pushpa|         USA|
|        priya|           pushpa|         USA|
|      Karthik|             Subu|         USA|
|        James|Smith            |         USA|
|       Martin|            Jones|         USA|
|          Sam| Anderson        |          UK|
|        Maria|          Patrick|          UK|
|        Robet|            Bevon|          UK|
|        Maria|         Anderson|          UK|
+-------------+-----------------+------------+



In [0]:
#option2 using the withcolumn renamed method
df1 = df.withColumnRenamed("firstname","F_N")\
        .withColumnRenamed("lastname","LN")
df1.show()        

+-------------+-----------------+------------+-----+-----+
|          F_N|               LN|     country|state|  sal|
+-------------+-----------------+------------+-----+-----+
|         raja|           pushpa|         USA|     |30000|
|        priya|           pushpa|         USA|     |29900|
|      Karthik|             Subu|         USA|   CA| 6000|
|        James|Smith            |         USA|   FL|20000|
|       Martin|            Jones|         USA|   CA| 3000|
|          Sam| Anderson        |          UK|  LND| 8000|
|        Maria|          Patrick|          UK|  MCR| 7000|
|        Robet|            Bevon|          UK|  LND| 3500|
|        Maria|         Anderson|          UK|  MCR| 3000|
+-------------+-----------------+------------+-----+-----+



#FILTER
We are selecting the required rows from a dataframe syntax: dataframe.filter("")



In [0]:
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.filter.html

from pyspark.sql.types import *
staticlist = [("raja", "pushpa", "USA","",30000),
            ("priya", "pushpa", "USA","",29900),
            ("Karthik", "Subu", "USA","CA",6000),
            ("James", "Smith", "USA","FL",20000),
            ("Martin", "Jones", "USA","CA",3000),
            ("Sam", "Anderson", "UK","LND",8000),
            ("Maria", "Patrick", "UK","MCR",7000),
            ("Robet", "Bevon", "UK","LND",3500),
            ("Maria", "Anderson", "UK","MCR",3000)
            ]
columns = ["firstname","lastname","country","state","sal"]            
df = spark.createDataFrame( data = staticlist, schema = columns)
df.show()

+---------+--------+-------+-----+-----+
|firstname|lastname|country|state|  sal|
+---------+--------+-------+-----+-----+
|     raja|  pushpa|    USA|     |30000|
|    priya|  pushpa|    USA|     |29900|
|  Karthik|    Subu|    USA|   CA| 6000|
|    James|   Smith|    USA|   FL|20000|
|   Martin|   Jones|    USA|   CA| 3000|
|      Sam|Anderson|     UK|  LND| 8000|
|    Maria| Patrick|     UK|  MCR| 7000|
|    Robet|   Bevon|     UK|  LND| 3500|
|    Maria|Anderson|     UK|  MCR| 3000|
+---------+--------+-------+-----+-----+



In [0]:
# place a filter where salary is above 6000 (single column filter)
df_filter1 = df.filter("sal >6000")
df_filter1.show()

+---------+--------+-------+-----+-----+
|firstname|lastname|country|state|  sal|
+---------+--------+-------+-----+-----+
|     raja|  pushpa|    USA|     |30000|
|    priya|  pushpa|    USA|     |29900|
|    James|   Smith|    USA|   FL|20000|
|      Sam|Anderson|     UK|  LND| 8000|
|    Maria| Patrick|     UK|  MCR| 7000|
+---------+--------+-------+-----+-----+



In [0]:
#multiple column filters
#get employees with salary > 6000 and contry is UK
df_filter2 = df.filter("sal > 6000 AND country = 'UK' and state = 'MCR' ")
df_filter2.show()

+---------+--------+-------+-----+----+
|firstname|lastname|country|state| sal|
+---------+--------+-------+-----+----+
|    Maria| Patrick|     UK|  MCR|7000|
+---------+--------+-------+-----+----+



In [0]:
# Multiple columns and multiple conditions
# if country is USA then salary > 6000, if country is not USA then salary > 3000

df_filter3 = df.filter( 
                       ((df.country == "USA") & (df.sal > 20000))
                     |
                       ((df.country != "USA") & (df.sal > 3000))
)

df_filter3.show()


+---------+--------+-------+-----+-----+
|firstname|lastname|country|state|  sal|
+---------+--------+-------+-----+-----+
|     raja|  pushpa|    USA|     |30000|
|    priya|  pushpa|    USA|     |29900|
|      Sam|Anderson|     UK|  LND| 8000|
|    Maria| Patrick|     UK|  MCR| 7000|
|    Robet|   Bevon|     UK|  LND| 3500|
+---------+--------+-------+-----+-----+



#String


##

In [0]:
# get the upper case, lower case and title case for any sting
# you can apply these functions inside the withcolumn method to apply the required transformations

df2 = df.withColumn("UpperCase",upper("firstname"))\
        .withColumn("LowerCase",lower("firstname"))\
        .withColumn("TitleCase",initcap("firstname"))    
df2.show()        


+---------+--------+-------+-----+-----+---------+---------+---------+
|firstname|lastname|country|state|  sal|UpperCase|LowerCase|TitleCase|
+---------+--------+-------+-----+-----+---------+---------+---------+
|     raja|  pushpa|    USA|     |30000|     RAJA|     raja|     Raja|
|    priya|  pushpa|    USA|     |29900|    PRIYA|    priya|    Priya|
|  Karthik|    Subu|    USA|   CA| 6000|  KARTHIK|  karthik|  Karthik|
|    James|   Smith|    USA|   FL|20000|    JAMES|    james|    James|
|   Martin|   Jones|    USA|   CA| 3000|   MARTIN|   martin|   Martin|
|      Sam|Anderson|     UK|  LND| 8000|      SAM|      sam|      Sam|
|    Maria| Patrick|     UK|  MCR| 7000|    MARIA|    maria|    Maria|
|    Robet|   Bevon|     UK|  LND| 3500|    ROBET|    robet|    Robet|
|    Maria|Anderson|     UK|  MCR| 3000|    MARIA|    maria|    Maria|
+---------+--------+-------+-----+-----+---------+---------+---------+



In [0]:
# cancat firstname and lastname to create a fullname
# concat ("col1","col2","col3")
df2 = df.withColumn("Full_Name",concat(initcap("firstname"),lit(" "),(initcap("lastname"))))
df2.show()

+---------+--------+-------+-----+-----+--------------+
|firstname|lastname|country|state|  sal|     Full_Name|
+---------+--------+-------+-----+-----+--------------+
|     raja|  pushpa|    USA|     |30000|   Raja Pushpa|
|    priya|  pushpa|    USA|     |29900|  Priya Pushpa|
|  Karthik|    Subu|    USA|   CA| 6000|  Karthik Subu|
|    James|   Smith|    USA|   FL|20000|   James Smith|
|   Martin|   Jones|    USA|   CA| 3000|  Martin Jones|
|      Sam|Anderson|     UK|  LND| 8000|  Sam Anderson|
|    Maria| Patrick|     UK|  MCR| 7000| Maria Patrick|
|    Robet|   Bevon|     UK|  LND| 3500|   Robet Bevon|
|    Maria|Anderson|     UK|  MCR| 3000|Maria Anderson|
+---------+--------+-------+-----+-----+--------------+



In [0]:
#removing leading and trailing spaces
#TRIM function will remove leading and trailing spaces from any string
df3 = df.withColumn("Tversion",trim("lastname"))
df3.show()

+---------+--------+-------+-----+-----+--------+
|firstname|lastname|country|state|  sal|Tversion|
+---------+--------+-------+-----+-----+--------+
|     raja|  pushpa|    USA|     |30000|  pushpa|
|    priya|  pushpa|    USA|     |29900|  pushpa|
|  Karthik|    Subu|    USA|   CA| 6000|    Subu|
|    James|   Smith|    USA|   FL|20000|   Smith|
|   Martin|   Jones|    USA|   CA| 3000|   Jones|
|      Sam|Anderson|     UK|  LND| 8000|Anderson|
|    Maria| Patrick|     UK|  MCR| 7000| Patrick|
|    Robet|   Bevon|     UK|  LND| 3500|   Bevon|
|    Maria|Anderson|     UK|  MCR| 3000|Anderson|
+---------+--------+-------+-----+-----+--------+



In [0]:
#get the left ,right and midle values of any given string
#syntax: substring("<colname>",<Starting_position>,<lenght_of_string>)
df4 =df.withColumn("Full_name",trim(concat(initcap("firstname"),lit(" "),initcap("lastname"))))
df5 = df4.withColumn("First4letters",substring("Full_name",0,4))\
         .withColumn("Last4letters",substring("Full_name",-4,4))\
         .withColumn("3rd_pos_4letters",substring("Full_name",3,4))\
          .withColumn("3rd_to_last",substring("Full_name",3,12))
df5.show()         

+---------+--------+-------+-----+-----+--------------+-------------+------------+----------------+------------+
|firstname|lastname|country|state|  sal|     Full_name|First4letters|Last4letters|3rd_pos_4letters| 3rd_to_last|
+---------+--------+-------+-----+-----+--------------+-------------+------------+----------------+------------+
|     raja|  pushpa|    USA|     |30000|   Raja Pushpa|         Raja|        shpa|            ja P|   ja Pushpa|
|    priya|  pushpa|    USA|     |29900|  Priya Pushpa|         Priy|        shpa|            iya |  iya Pushpa|
|  Karthik|    Subu|    USA|   CA| 6000|  Karthik Subu|         Kart|        Subu|            rthi|  rthik Subu|
|    James|   Smith|    USA|   FL|20000|   James Smith|         Jame|        mith|            mes |   mes Smith|
|   Martin|   Jones|    USA|   CA| 3000|  Martin Jones|         Mart|        ones|            rtin|  rtin Jones|
|      Sam|Anderson|     UK|  LND| 8000|  Sam Anderson|         Sam |        rson|            m 